In [4]:
import sys
sys.path.append('../..')
sys.path.append('../../APDFT')
sys.path.append('../../helper_code')
sys.path.append('../data')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from copy import deepcopy
from IPython.display import display
# from qml.representations import generate_coulomb_matrix, generate_fchl_acsf, generate_bob
from helper_code.util import charge_arr_to_str
from helper_code.MBDF import generate_mbdf, generate_df

In [5]:
coronene_energy_raw_data = np.load("../data/coronene_raw_data/coronene_mutants_pbe_pcx2_corrected3.npz", allow_pickle=True)
charges, total_energy = coronene_energy_raw_data['charges'], coronene_energy_raw_data['energies']

num_mol = charges.shape[0]
print(num_mol)

2043


In [6]:
xyz_file = '../data/coronene_raw_data/coronene_opt_pbe0_pcX2.xyz'
with open(xyz_file, 'r') as f:
    lines = f.readlines()[2:]  # Skip the first two lines (atom count and comment)
    atoms = [line.split() for line in lines]

# Extract atomic symbols and coordinates
symbols = [atom[0] for atom in atoms]
all_atoms_coord = [[float(atom[1]), float(atom[2]), float(atom[3])] for atom in atoms]
coordinates = np.tile(np.array(all_atoms_coord), (num_mol, 1, 1))

In [7]:
print(coordinates.shape)

(2043, 36, 3)


## CM, BOB, FCHL ##

In [16]:
# cm = np.array([generate_coulomb_matrix(q, r, size=36) for q, r, in zip(charges, coordinates)])
# bob = np.array([generate_bob(q, r, atomtypes=['C', 'N', 'B', 'H'], asize={"C":24, "N":12, "B":12, "H":12}) for q, r, in zip(charges, coordinates)])

In [17]:
# print(cm.shape)
# print(bob.shape)

(2400, 666)
(2400, 1830)
(2400, 36, 496)


In [19]:
# np.savetxt("../data/coronene_training_data/CM.csv", cm, delimiter=',')
# np.savetxt("../data/coronene_training_data/BOB.csv", bob, delimiter=',')

## MBDF ##

In [8]:
mbdf = generate_mbdf(charges, coordinates, local=True)
print(mbdf.shape) # (2043, 36, 6)

(2043, 36, 6)


In [9]:
df = generate_df(mbdf, charges)
print(df.shape) # (2043, 2400)

(2043, 2400)


In [10]:
mbdf_flatten = mbdf.reshape((mbdf.shape[0], -1))
np.savetxt("../data/coronene_training_data/MBDF.csv", mbdf_flatten, delimiter=',')
np.savetxt("../data/coronene_training_data/DF.csv", df, delimiter=',')